## Loading Modules

In [ ]:
import os
import pandas as pd
from matplotlib import pyplot as plt

from synapticTrack.io import BeamDataIOManager
from synapticTrack.visualizations import *
from synapticTrack.analysis import *

### Load Track Simulation Data

In [ ]:
base_dir = 'run.00'

filename = os.path.join(base_dir, 'beam.out')
df_beam = pd.read_csv(filename, sep=r'\s+', engine='python')

filename = os.path.join(base_dir, 'step.out')
df_step = pd.read_csv(filename, sep=r'\s+', engine='python')

In [ ]:
z_scanner = []
with open(os.path.join(base_dir, 'z_scanner.out'), 'r') as f:
    for line in f:
        z_scanner.append(line.strip())
print(z_scanner)

### TRACK Simulation Data

In [ ]:
coord_list = ['coord_wire_scanner1.out', 'coord_allison_scanner.out', 'coord_wire_scanner2.out', 'coord_wire_scanner3.out', 
              'coord_wire_scanner4.out', 'coord_ends.out']
beam_centroid = pd.DataFrame(columns=['z', 'x', 'xp', 'y', 'yp', 'dt', 'dW'])
beam_rms_size = pd.DataFrame(columns=['z', 'x', 'xp', 'y', 'yp', 'dt', 'dW'])

for i, file in enumerate(coord_list):
    filename = os.path.join(base_dir, file)

    beam_io_manager = BeamDataIOManager() 
    beam = beam_io_manager.read(code='track', filename=filename, mass_number=40, charge_state=8, beam_current=0, reference_energy=0.10)
    beam_centroid.loc[len(beam_centroid)] = beam.centroid
    beam_rms_size.loc[len(beam_rms_size)] = beam.rms_size
    beam_centroid.loc[i, 'z'] = float(z_scanner[i])*10
    beam_rms_size.loc[i, 'z'] = float(z_scanner[i])*10

# Load wire scanner offsets
ws_offset_x = np.load("ws_offset_x.npy")
ws_offset_y = np.load("ws_offset_y.npy")

beam_centroid_new = pd.DataFrame(columns=['z', 'x', 'y'])

# Apply beam centroid offsets at wire scanners
for i, ws in enumerate([0, 2, 3, 4]):
    beam_centroid_new.loc[i, 'z'] = beam_centroid.loc[ws, 'z']
    beam_centroid_new.loc[i, 'x'] = beam_centroid.loc[ws, 'x'] - ws_offset_x[i]
    beam_centroid_new.loc[i, 'y'] = beam_centroid.loc[ws, 'y'] - ws_offset_y[i]

print('beam centroid')
print(beam_centroid)

print('beam centroid_new')
print(beam_centroid_new)

print('beam rms size')
print(beam_rms_size)

### Scanner Measurement Data

In [ ]:
wire_base_dir = '../data/scanner/2_exp_LEBT_WS'
allison_base_dir = '../data/scanner/3_exp_Allison'
scanner_file_list = ['ECR32-WS001-100717.txt', ['101614_X.txt', '102829_Y.txt'], 
                     'LEBT-WS002-100523.txt', 'LEBT-WS003-100325.txt', 'LEBT-WS004-100044.txt']

skip_as = True
scanner_centroid = pd.DataFrame(columns=['z', 'x', 'y', 'x_fit', 'y_fit'])
scanner_rms_size = pd.DataFrame(columns=['z', 'x', 'y'])

for i, scanner_file in enumerate(scanner_file_list):
    beam_io_manager = BeamDataIOManager() 
    if i == 1:
        filename0 = os.path.join(allison_base_dir, scanner_file[0])
        beamas0 = beam_io_manager.read_scanner(scanner='allison', filename=filename0)
        AS0 = analyze_allison_scanner_2d(beamas0, plot=False)
        scanner_centroid.loc[i, 'z'] = float(z_scanner[i])*10
        scanner_centroid.loc[i, 'x'] = AS0['x_center']
        scanner_centroid.loc[i, 'x_fit'] = AS0['gaussian_fit_x_center']
        scanner_rms_size.loc[i, 'z'] = float(z_scanner[i])*10
        scanner_rms_size.loc[i, 'x'] = AS0['sigma_x']

        filename1 = os.path.join(allison_base_dir, scanner_file[1])
        beamas1 = beam_io_manager.read_scanner(scanner='allison', filename=filename1)
        AS1 = analyze_allison_scanner_2d(beamas1, plot=False)
        scanner_centroid.loc[i, 'z'] = float(z_scanner[i])*10
        scanner_centroid.loc[i, 'y'] = AS1['x_center']
        scanner_centroid.loc[i, 'y_fit'] = AS1['gaussian_fit_x_center']
        scanner_rms_size.loc[i, 'z'] = float(z_scanner[i])*10
        scanner_rms_size.loc[i, 'y'] = AS1['sigma_x']
    else:
        filename = os.path.join(wire_base_dir, scanner_file)
        beamws = beam_io_manager.read_scanner(scanner='wire', filename=filename)
        WS = analyze_wire_scanner(beamws, plot=False)
        scanner_centroid.loc[i, 'z'] = float(z_scanner[i])*10
        scanner_centroid.loc[i, 'x'] = WS['x_center']
        scanner_centroid.loc[i, 'y'] = WS['y_center']
        scanner_centroid.loc[i, 'x_fit'] = WS['gaussian_fit_x_center']
        scanner_centroid.loc[i, 'y_fit'] = WS['gaussian_fit_y_center']
        scanner_rms_size.loc[i, 'z'] = float(z_scanner[i])*10
        scanner_rms_size.loc[i, 'x'] = WS['sigma_x']
        scanner_rms_size.loc[i, 'y'] = WS['sigma_y']
print("scanner centroid")
print(scanner_centroid)
print("scanner rms size")
print(scanner_rms_size)

### Beam Centroid Plot

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Horizontal Beam Centroid vs. z")
#plt.plot(df_beam['dist[m]']*1e3, df_beam['Xc[cm]']*10, c='k', label='beam.out')
plt.plot(df_step['z[cm]']*10, df_step['X-avg[cm]']*10, c='b', label='step.out')
plt.scatter(beam_centroid['z'], beam_centroid['x'], c='orange', label='track', marker='x', s=30)
plt.scatter(beam_centroid_new['z'], beam_centroid_new['x'], c='magenta', label='track_calibrated', marker='v', s=30)
plt.scatter(scanner_centroid['z'], scanner_centroid['x'], c='r', label='measurement', marker='^', s=30)
#plt.scatter(scanner_centroid['z'], scanner_centroid['x_fit'], c='g', label='measurement_fit', marker='o')
plt.axhline(y=0.0, c='g', ls='--')
plt.xlabel(r"$z$ [mm]", fontsize=16)
plt.ylabel(r"$x_\text{c}$ [mm]", fontsize=16)
plt.legend(loc='best')
plt.grid()

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Vertical Beam Centroid vs. z")
#plt.plot(df_beam['dist[m]']*1e3, df_beam['Yc[cm]']*10, c='k', label='beam.out')
plt.plot(df_step['z[cm]']*10, df_step['Y-avg[cm]']*10, c='b', label='step.out')
plt.scatter(beam_centroid['z'], beam_centroid['y'], c='orange', label='track', marker='x', s=30)
plt.scatter(beam_centroid_new['z'], beam_centroid_new['y'], c='magenta', label='track_calibrated', marker='v', s=30)
plt.scatter(scanner_centroid['z'], scanner_centroid['y'], c='r', label='measurement', marker='^', s=30)
#plt.scatter(scanner_centroid['z'], scanner_centroid['y_fit'], c='g', label='measurement_fit', marker='o')
plt.axhline(y=0.0, c='g', ls='--')
plt.xlabel(r"$z$ [mm]", fontsize=16)
plt.ylabel(r"$y_\text{c}$ [mm]", fontsize=16)
plt.legend(loc='best')
plt.grid()

### Beam RMS Size Plot

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Horizontal RMS Beam Size vs. z")
#plt.plot(df_beam['dist[m]']*1e3, df_beam['x_rms[cm]']*10, c='k', label='beam.out')
plt.plot(df_step['z[cm]']*10, df_step['X-rms[cm]']*10, c='b', label='step.out')
plt.scatter(beam_rms_size['z'], beam_rms_size['x'], c='orange', label='track', marker='x', s=30)
plt.scatter(scanner_rms_size['z'], scanner_rms_size['x'], c='r', label='measurement', marker='v', s=30)
plt.xlabel(r"$z$ [mm]", fontsize=16)
plt.ylabel(r"$x_{\text{rms}}$ [mm]", fontsize=16)
plt.legend(loc='best')
plt.grid()

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Vertical RMS Beam Size vs. z")
#plt.plot(df_beam['dist[m]']*1e3, df_beam['y_rms[cm]']*10, c='k', label='beam.out')
plt.plot(df_step['z[cm]']*10, df_step['Y-rms[cm]']*10, c='b', label='step.out')
plt.scatter(beam_rms_size['z'], beam_rms_size['y'], c='orange', label='track', marker='x')
plt.scatter(scanner_rms_size['z'], scanner_rms_size['y'], c='r', label='maasurement', marker='+')
plt.xlabel(r"$z$ [mm]", fontsize=16)
plt.ylabel(r"$y_{\text{rms}}$ [mm]", fontsize=16)
plt.legend(loc='best')
plt.grid()